In [1]:
# Importing all necessery libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# checking whether GPU has been intialised or not
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cpu


In [2]:
# Creating Model

drop_out_val = 0.05 # seting dropout as 5 %

class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # CONVOLUTION LAYER 1                     
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=(3,3), padding=1)      
        self.bn1 = nn.BatchNorm2d(12)
        self.do1 = nn.Dropout2d(drop_out_val)
        
        # CONVOLUTION LAYER 2
        self.conv2 = nn.Conv2d(12, 24, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(24)
        self.do2 = nn.Dropout2d(drop_out_val)
        
        # CONVOLUTION LAYER 3
        self.conv3  = nn.Conv2d(24, 36, 3, padding=1)
        self.bn3    = nn.BatchNorm2d(36)
        self.do3    = nn.Dropout2d(drop_out_val)

        # TRANSITION LAYER 1
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv_t1 = nn.Conv2d(36, 12, 1)
        self.bn_t1 = nn.BatchNorm2d(12)
        self.do_t1 = nn.Dropout2d(drop_out_val)

        # CONVOLUTION LAYER 4
        self.conv4 = nn.Conv2d(12, 12, 3, padding = 1)
        self.bn4 = nn.BatchNorm2d(12)
        self.do4 = nn.Dropout2d(drop_out_val)

        # CONVOLUTION LAYER 5
        self.conv5  = nn.Conv2d(12, 24, 3, padding = 1)
        self.bn5    = nn.BatchNorm2d(24)
        self.do5    = nn.Dropout2d(drop_out_val)


        # TRANSITION LAYER 2
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv_t2  = nn.Conv2d(24, 12, 1)
        self.bn_t2    = nn.BatchNorm2d(12)
        self.do_t2    = nn.Dropout2d(drop_out_val)

        # CONVOLUTION LAYER 6
        self.conv6 = nn.Conv2d(12, 12, 3, padding = 1)
        self.bn6 = nn.BatchNorm2d(12)
        self.do6 = nn.Dropout2d(drop_out_val)

        # CONVOLUTION LAYER 7
        self.conv7 = nn.Conv2d(12, 24, 3, padding = 1)

        self.gap = nn.AvgPool2d(7)
        self.linear1 = nn.Linear(24, 10)

    def forward(self, x):
        #BN after Relu
        x = self.pool1(self.do3(self.bn3(F.relu(self.conv3 (self.do2(self.bn2(F.relu(self.conv2  (self.do1(self.bn1(F.relu(self.conv1(x)))))))))))))  
        x = self.pool2(self.do5(self.bn5(F.relu(self.conv5(self.do4(self.bn4(self.conv4((self.do_t1(self.bn_t1(F.relu(self.conv_t1(x)))))))))))))
        x = F.relu( self.conv7(self.do6(self.bn6(F.relu(self.conv6 (self.do_t2(self.bn_t2(F.relu(self.conv_t2(x))))))))))               
        x = self.gap(x)
        x = x.view(-1, 24)
        x = self.linear1(x)

        return F.log_softmax(x)       #log of softmax


model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 28, 28]             120
       BatchNorm2d-2           [-1, 12, 28, 28]              24
         Dropout2d-3           [-1, 12, 28, 28]               0
            Conv2d-4           [-1, 24, 28, 28]           2,616
       BatchNorm2d-5           [-1, 24, 28, 28]              48
         Dropout2d-6           [-1, 24, 28, 28]               0
            Conv2d-7           [-1, 36, 28, 28]           7,812
       BatchNorm2d-8           [-1, 36, 28, 28]              72
         Dropout2d-9           [-1, 36, 28, 28]               0
        MaxPool2d-10           [-1, 36, 14, 14]               0
           Conv2d-11           [-1, 12, 14, 14]             444
      BatchNorm2d-12           [-1, 12, 14, 14]              24
        Dropout2d-13           [-1, 12, 14, 14]               0
           Conv2d-14           [-1, 12,

<ipython-input-2-b411df3ebd3f>:69: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)       #log of softmax


In [3]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9912422/9912422 [00:04<00:00, 2365546.58it/s]


Extracting ../data\MNIST\raw\train-images-idx3-ubyte.gz to ../data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 3279166.61it/s]


Extracting ../data\MNIST\raw\train-labels-idx1-ubyte.gz to ../data\MNIST\raw



100%|██████████| 1648877/1648877 [00:01<00:00, 1011012.76it/s]


Extracting ../data\MNIST\raw\t10k-images-idx3-ubyte.gz to ../data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ../data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../data\MNIST\raw



In [4]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [5]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 10):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-2-b411df3ebd3f>:69: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)       #log of softmax
loss=0.3610001504421234 batch_id=0:   0%|          | 1/469 [00:00<01:04,  7.31it/s]


Test set: Average loss: 0.1467, Accuracy: 9575/10000 (96%)



loss=0.18494072556495667 batch_id=0:   0%|          | 1/469 [00:00<01:06,  6.99it/s]


Test set: Average loss: 0.0731, Accuracy: 9781/10000 (98%)



loss=0.13285934925079346 batch_id=0:   0%|          | 1/469 [00:00<01:12,  6.48it/s]

...


Test set: Average loss: 0.0388, Accuracy: 9882/10000 (99%)



loss=0.06597888469696045 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0351, Accuracy: 9889/10000 (99%)



loss=0.09651247411966324 batch_id=468: 100%|██████████| 469/469 [01:22<00:00,  5.71it/s] 



Test set: Average loss: 0.0331, Accuracy: 9899/10000 (99%)

